In [ ]:
!pip install tensorflowjs

In [ ]:
import numpy as np
import pandas as pd
import os


from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import r2_score

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!unzip -u "/content/drive/MyDrive/Weather Prediction - Data and Results.zip" -d "/content/drive/MyDrive/Weather Prediction - Data and Results" 

Mounted at /content/drive
Archive:  /content/drive/MyDrive/Weather Prediction - Data and Results.zip
  inflating: /content/drive/MyDrive/Weather Prediction - Data and Results/Weather Prediction - Data and Results/results.csv  
   creating: /content/drive/MyDrive/Weather Prediction - Data and Results/Weather Prediction - Data and Results/model/assets/
  inflating: /content/drive/MyDrive/Weather Prediction - Data and Results/Weather Prediction - Data and Results/2022-03-03-16-44-40.jpg  
  inflating: /content/drive/MyDrive/Weather Prediction - Data and Results/Weather Prediction - Data and Results/2022-03-03-19-13-04.jpg  
  inflating: /content/drive/MyDrive/Weather Prediction - Data and Results/Weather Prediction - Data and Results/2022-03-04-12-20-25.jpg  
  inflating: /content/drive/MyDrive/Weather Prediction - Data and Results/Weather Prediction - Data and Results/2022-03-04-14-38-43.jpg  
  inflating: /content/drive/MyDrive/Weather Prediction - Data and Results/Weather Prediction - 

In [ ]:
image_dir = "./drive/MyDrive/Weather Prediction - Data and Results/Weather Prediction - Data and Results/"

In [ ]:
df =  pd.read_csv(image_dir + "temps.csv")

mask = df["name"] == "Mita"
df = df[mask].reset_index(drop=True)
# df = df.sample(frac=1).reset_index(drop=True)

df  = df.rename(columns={"time":"path", "main_temp":"temp" })
df["path"] = image_dir + df["path"].astype(str) + ".jpg"
df = df[["path", "temp"]]
# df = df.sample(100, random_state=1).reset_index(drop=True)
df

,path,temp
0,./drive/MyDrive/Weather Prediction - Data and ...,9.52
1,./drive/MyDrive/Weather Prediction - Data and ...,7.94
2,./drive/MyDrive/Weather Prediction - Data and ...,11.24
3,./drive/MyDrive/Weather Prediction - Data and ...,9.97
4,./drive/MyDrive/Weather Prediction - Data and ...,5.92
...,...,...
1897,./drive/MyDrive/Weather Prediction - Data and ...,8.23
1898,./drive/MyDrive/Weather Prediction - Data and ...,11.08
1899,./drive/MyDrive/Weather Prediction - Data and ...,7.87
1900,./drive/MyDrive/Weather Prediction - Data and ...,13.53


In [ ]:
train_df, test_df = train_test_split(df, train_size=0.7, shuffle=True, random_state=1)

In [ ]:
with tf.device('/device:GPU:0'):
  train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
  )

  test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale=1./255
  )

In [ ]:
with tf.device('/device:GPU:0'):
  train_images = train_generator.flow_from_dataframe(
      dataframe=train_df,
      x_col='path',
      y_col='temp',
      target_size=(224, 224),
      color_mode='rgb',
      class_mode='raw',
      batch_size=32,
      shuffle=True,
      seed=42,
      subset='training'
  )

  val_images = train_generator.flow_from_dataframe(
      dataframe=train_df,
      x_col='path',
      y_col='temp',
      target_size=(224, 224),
      color_mode='rgb',
      class_mode='raw',
      batch_size=32,
      shuffle=True,
      seed=42,
      subset='validation'
  )

  test_images = test_generator.flow_from_dataframe(
      dataframe=test_df,
      x_col='path',
      y_col='temp',
      target_size=(224, 224),
      color_mode='rgb',
      class_mode='raw',
      batch_size=32,
      shuffle=False
  )

Found 1065 validated image filenames.
Found 266 validated image filenames.
Found 571 validated image filenames.


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import Input, DepthwiseConv2D, MaxPooling2D
from keras.layers import Conv2D, BatchNormalization,Dropout,concatenate
from keras.layers import ReLU, AvgPool2D, Flatten, Dense,Activation
from keras.layers.wrappers import TimeDistributed
from keras.applications.mobilenet import MobileNet
from keras import Model
from keras.models import Model,Sequential, load_model
import keras.backend as K
import tensorflow.keras.backend as KB


from keras.preprocessing import image as iimg
import tensorflow.keras.layers as kl
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
#import tensorflow_addons as tfa

from sklearn.decomposition import PCA
import pickle as pk
from tensorflow.compat.v1 import ConfigProto, InteractiveSession
import datetime


In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.densenet import DenseNet169,DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50 
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.applications.resnet_v2 import ResNet50V2 
from tensorflow.keras.applications import MobileNetV2, ResNet152V2,MobileNet
from tensorflow.keras.applications import Xception,InceptionResNetV2	
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
with tf.device('/device:GPU:0'):
  inputs = tf.keras.Input(shape=(224, 224, 3))

  mod = VGG16(input_shape=(224, 224, 3),include_top=False)
  for layer in mod.layers:
    layer.trainable = False
  x = mod(inputs)
  x = Conv2D(128, 1, activation="linear", name = "Image_Conv_1")(x)
  x = tf.keras.layers.MaxPool2D()(x)
  x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='linear')(x)
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)
  x = tf.keras.layers.Dense(64, activation='relu')(x)
  outputs = tf.keras.layers.Dense(1, activation='linear')(x)


  '''x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
  x = tf.keras.layers.MaxPool2D()(x)
  x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
  x = tf.keras.layers.MaxPool2D()(x)
  '''

  model = tf.keras.Model(inputs=inputs, outputs=outputs)

  optimizer= tf.keras.optimizers.Adam(learning_rate=1e-3)
  model.compile(loss= 'mse', optimizer=optimizer)


  history = model.fit(
      train_images,
      validation_data=val_images,
      epochs=100,
      callbacks=[
          tf.keras.callbacks.EarlyStopping(
              monitor='val_loss',
              patience=10,
              restore_best_weights=True
          )
      ]
  )

Epoch 1/100
34/34 [==============================] - 8s 206ms/step - loss: 21.3507 - val_loss: 4.7010
Epoch 2/100
34/34 [==============================] - 7s 204ms/step - loss: 3.7997 - val_loss: 2.6630
Epoch 3/100
34/34 [==============================] - 7s 204ms/step - loss: 1.7463 - val_loss: 1.5081
Epoch 4/100
34/34 [==============================] - 7s 206ms/step - loss: 0.8577 - val_loss: 0.7672
Epoch 5/100
34/34 [==============================] - 7s 206ms/step - loss: 0.5393 - val_loss: 0.5563
Epoch 6/100
34/34 [==============================] - 7s 205ms/step - loss: 0.3638 - val_loss: 0.3871
Epoch 7/100
34/34 [==============================] - 7s 204ms/step - loss: 0.2695 - val_loss: 0.2893
Epoch 8/100
34/34 [==============================] - 7s 202ms/step - loss: 0.2335 - val_loss: 0.6165
Epoch 9/100
34/34 [==============================] - 7s 203ms/step - loss: 0.3046 - val_loss: 0.3302
Epoch 10/100
34/34 [==============================] - 7s 201ms/step - loss: 0.2013 - val_l

In [ ]:
with tf.device('/device:GPU:0'):
  preds_test = np.squeeze(model.predict(test_images))
  trues_test = test_images.labels

  rmse = np.sqrt(model.evaluate(test_images, verbose=0))
  print("     Test RMSE: {:.5f}".format(rmse))

  r2 = r2_score(trues_test, preds_test)
  print("Test R^2 Score: {:.5f}".format(r2))

     Test RMSE: 0.23655
Test R^2 Score: 0.99117


In [ ]:

print('mean_True_Test : ', np.mean(trues_test))
print('mean_Pred_Test : ', np.mean(preds_test))

mean_True_Test :  10.118056042031524
mean_Pred_Test :  10.120415


In [ ]:
with tf.device('/device:GPU:0'):
  all_images = train_generator.flow_from_dataframe(
      dataframe=df,
      x_col='path',
      y_col='temp',
      target_size=(224, 224),
      color_mode='rgb',
      class_mode='raw',
      batch_size=32,
      shuffle=False)
  
with tf.device('/device:GPU:0'):
  preds = np.squeeze(model.predict(all_images))
  trues = all_images.labels

Found 1902 validated image filenames.


In [ ]:
df['preds'] = pd.Series(preds)
df['trues'] = pd.Series(trues)
df

,path,temp,preds,trues
0,./drive/MyDrive/Weather Prediction - Data and ...,9.52,9.361343,9.52
1,./drive/MyDrive/Weather Prediction - Data and ...,7.94,8.046840,7.94
2,./drive/MyDrive/Weather Prediction - Data and ...,11.24,11.197207,11.24
3,./drive/MyDrive/Weather Prediction - Data and ...,9.97,9.877302,9.97
4,./drive/MyDrive/Weather Prediction - Data and ...,5.92,6.177597,5.92
...,...,...,...,...
1897,./drive/MyDrive/Weather Prediction - Data and ...,8.23,8.140255,8.23
1898,./drive/MyDrive/Weather Prediction - Data and ...,11.08,10.897459,11.08
1899,./drive/MyDrive/Weather Prediction - Data and ...,7.87,7.764625,7.87
1900,./drive/MyDrive/Weather Prediction - Data and ...,13.53,13.693209,13.53


In [ ]:
df.to_csv("./drive/MyDrive/Weather Prediction - Data and Results/Weather Prediction - Data and Results/results.csv", index=False)

In [ ]:
model.save("./drive/MyDrive/Weather Prediction - Data and Results/Weather Prediction - Data and Results/model")

INFO:tensorflow:Assets written to: ./drive/MyDrive/Weather Prediction - Data and Results/Weather Prediction - Data and Results/model/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/Weather Prediction - Data and Results/Weather Prediction - Data and Results/model/assets


#### Save model for web inference

In [ ]:
import tensorflowjs as tfjs

     |████████████████████████████████| 511.7 MB 6.1 kB/s 
     |████████████████████████████████| 1.6 MB 49.1 MB/s 
     |████████████████████████████████| 5.8 MB 58.6 MB/s 
     |████████████████████████████████| 438 kB 46.3 MB/s 
     |████████████████████████████████| 511.7 MB 4.9 kB/s 
     |████████████████████████████████| 5.8 MB 31.3 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packagin

In [ ]:
tfjs.converters.save_keras_model(model, "./drive/MyDrive/Weather Prediction - Data and Results/Weather Prediction - Data and Results/tfjs_model")